## Use bonus plates (length of 4)
## Important notes:
### - new color model: number of channels = 4, one channel for each color  
### - plate value encoded with a number: 0.1, 0.4, 1.0  
### - v3.2: all helpers moved to external module
### - v3.2: hint restored: s_before is now field after 'make_move'
### - v3.3: debugging CNN since learning doesn't work

In [1]:
# Needed for tests and a real game against the phone
#import pandas as pd
#import qgrid

import random as rd
import numpy as np
import tensorflow as tf

from collections import deque

import environment as ae

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Constants

In [2]:
# CNN
LEARNING_RATE = 0.01
UPDATE_TARGET_NET = 1000
MINIBATCH_SIZE = 64
GAMMA = 0.99

# Definitions
GAMES_TO_PLAY = 131072
REPLAY_MEMORY_SIZE = 131072
DYNAMIC_LEARNING_EPOCHS = 5
NUMBER_OF_MOVES_IN_GAME = 50

# Variables
MAXIMUM_SCORE = 0
CNN_MOVE_PROB = 0.1
CNN_MOVES_COUNT = 0
CNN_SUCCESSFUL_PREDICTION = 0
TOTAL_SCORE_500 = 0.0
TOTAL_SUCCESSFUL_MOVES_500 = 0.0

### Replay Memory Buffer

In [3]:
#
# Replay memory buffer
#
class ExperienceBuffer():
    '''
    Experience Replay Buffer
    Inspired by Andrea Lonza
    '''

    def __init__(self, buffer_size, gamma):
        # Constants
        self.gamma = gamma
        
        # Main Replay Memory buffer parts
        self.states_before = deque(maxlen=buffer_size)
        self.actions = deque(maxlen=buffer_size)
        self.total_rewards = deque(maxlen=buffer_size)
        self.states_after = deque(maxlen=buffer_size)
        self.last_moves = deque(maxlen=buffer_size)
   
    
    def add(self, state_before, action, reward, state_after, last_move):
        # Add certain items to corresponding buffers
        self.states_before.append(state_before)
        self.actions.append(action)
        self.total_rewards.append(reward)
        self.states_after.append(state_after)
        self.last_moves.append(last_move)
    
    
    def sample_minibatch(self, minibatch_size):
        '''
        Sample a minibatch of size batch_size
        Note1: always add the most recent completed move
        '''
        indices = rd.sample(range(len(self.states_before) - 1), minibatch_size - 1)
        # Add the most recent completed move index
        indices.append(len(self.states_before) - 1)
        
        minibatch_states_before = np.array([self.states_before[i] for i in indices]) 
        minibatch_actions = np.array([self.actions[i] for i in indices]) 
        minibatch_total_rewards = np.array([self.total_rewards[i] for i in indices]) 
        minibatch_states_after = np.array([self.states_after[i] for i in indices])  
        minibatch_last_moves = np.array([self.last_moves[i] for i in indices])   
        
        return minibatch_states_before, minibatch_actions, minibatch_total_rewards, minibatch_states_after, minibatch_last_moves
    
    
    def __len__(self):
        '''
        Return length of the current replay memory buffer
        Relevant for the first *minibatch_size* moves.
        '''
        return len(self.states_before)
    
    

## Deep Q-Nework

### Initialize Replay Buffer

In [4]:
replay_memory = ExperienceBuffer(REPLAY_MEMORY_SIZE, GAMMA)

### Initialize Online CNN and Target CNN

In [5]:
# Online CNN
Online_CNN = tf.keras.models.Sequential()
Online_CNN.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh, data_format = 'channels_last', input_shape=(7, 6, 4)))
Online_CNN.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh))    
Online_CNN.add(tf.keras.layers.Flatten())                      
Online_CNN.add(tf.keras.layers.Dense(ae.ACTIONS_DIMENSION, activation=None, kernel_initializer='RandomNormal'))

# Target CNN
Target_CNN = tf.keras.models.Sequential()
Target_CNN.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh, data_format = 'channels_last', input_shape=(7, 6, 4)))
Target_CNN.add(tf.keras.layers.Conv2D(192, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh))    
Target_CNN.add(tf.keras.layers.Flatten())                      
Target_CNN.add(tf.keras.layers.Dense(ae.ACTIONS_DIMENSION, activation=None, kernel_initializer='RandomNormal'))

# Initialize optimizer
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

# Set weights equal
Target_CNN.set_weights(Online_CNN.get_weights())

In [6]:
# Number of moves made to follow the target CNN update strategy
total_moves = 1

In [7]:
def loss(model, X, y_true, A):
    prediction = model(X)
    selected_action_values = tf.math.reduce_sum(prediction*A, axis=1)  
    return tf.keras.losses.MSE(y_true, selected_action_values)


def grad(model, inputs, targets, actions):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, actions)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

## Dynamic Learning

In [8]:
%%time
                                                      
for game in range(GAMES_TO_PLAY):
    # Start one game
    game_score = 0
    successful_moves = 0

    # Initialize the game field
    field = np.zeros((7, 6, 4))
    field = ae.initialize_field_3D(field)

    for m in range(NUMBER_OF_MOVES_IN_GAME):
        # Total score of one move
        reward = 0

        # Whether CNN made the move
        cnn_made_move_flag = False
        
        # If replay_memory has less than 64 moves, then make a random move
        if ((len(replay_memory) < MINIBATCH_SIZE) or (rd.random() > CNN_MOVE_PROB)):
            move = rd.randint(1, ae.ACTIONS_DIMENSION)
        else:
            # CNN selects a move
            cnn_made_move_flag = True
            CNN_MOVES_COUNT = CNN_MOVES_COUNT + 1
            _, move = ae.predict_max_score_3D(field.copy(), Target_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)

        # Make the move
        new_field, plate_a, plate_b = ae.make_move_v2_3D(field.copy(), move, ae.MOVES)
        field_after_move = new_field.copy()

        # Calculate the score and update the field
        score, new_field = ae.calculate_score_v2_3D(new_field, plate_a, plate_b)
        
        # If the move is successful, then update the field and check if we have new sets        
        successful_move_flag = False

        # While we have new sets (thus the score is greater than 0), process them, calculate score and move plates
        while (score > 0.):
            if (not successful_move_flag):
                successful_moves = successful_moves + 1
                successful_move_flag = True

            # Add new points to the total score of the move
            reward = reward + score

            # Move plates downward, fill the upper row so, that it doesn't have "easy" sets of three
            # Start from the left lower corner (in order to reuse color_fits())
            new_field = ae.fill_field_3D(new_field)

            # Calculate score and check whether we have new sets
            score, new_field = ae.calculate_score_v2_3D(new_field, (-1, -1), (-1, -1))

        # Increase the score of the whole game
        game_score = game_score + reward
        
        # Check whether it's the last move of the current game
        last_move = m == NUMBER_OF_MOVES_IN_GAME - 1         

        # Add new move to the replay memory
        if (successful_move_flag):
            replay_memory.add(field_after_move, move, reward, new_field, last_move)
            
            # Update CNN move statistics
            if (cnn_made_move_flag):    
                CNN_SUCCESSFUL_PREDICTION = CNN_SUCCESSFUL_PREDICTION + 1               
                
        else:
            replay_memory.add(field, move, 0, field, last_move)   
        
        #
        # Train CNN based on the score
        #
        if (len(replay_memory) >= MINIBATCH_SIZE):
            # Select random MINIBATCH_SIZE moves from replay memory buffer
            samples = replay_memory.sample_minibatch(MINIBATCH_SIZE)

            # Prepare some things for training
            s_before = samples[0]
            actions = samples[1]
            rewards = samples[2]
            s_after = samples[3] 
            dones = samples[4]
            
            rewards_next = np.max(Target_CNN(s_after), axis=1)
            actual_values = np.where(dones, rewards, rewards + GAMMA*rewards_next)
            selected_actions = tf.one_hot(actions, ae.ACTIONS_DIMENSION)
            
            # Update online CNN weights: training step
            for _ in range(DYNAMIC_LEARNING_EPOCHS):             
                loss_value, grads = grad(Online_CNN, s_before, actual_values, selected_actions)
                optimizer.apply_gradients(zip(grads, Online_CNN.trainable_variables))

        # If move is successful, update the play field
        if (successful_move_flag):
            field = np.copy(new_field)
            
        # After each 1000 moves update target CNN
        if (total_moves % UPDATE_TARGET_NET == 0):
            Target_CNN.set_weights(Online_CNN.get_weights())
            
        total_moves = total_moves + 1

    #
    # Calculate and display overall stats
    #
    # Check whether we have new maximum score
    if (game_score > MAXIMUM_SCORE):
        print(f"New maximum: {game_score}, after {game} games.")
        MAXIMUM_SCORE = game_score
        
    # After each 500 games output average game score, average number of successful moves per game
    TOTAL_SCORE_500 = TOTAL_SCORE_500 + game_score
    TOTAL_SUCCESSFUL_MOVES_500 = TOTAL_SUCCESSFUL_MOVES_500 + successful_moves
    
    if ((game % 500 == 0) and (game > 0)):
        avg_score = TOTAL_SCORE_500 / 500
        TOTAL_SCORE_500 = 0.0
        
        avg_succ_moves = TOTAL_SUCCESSFUL_MOVES_500 / 500
        TOTAL_SUCCESSFUL_MOVES_500 = 0.0

        print(f"Games: {game}, last 500 games avg score: {avg_score}, avg of successful moves: {avg_succ_moves}, loss {loss_value.numpy()}")        
        print(f"CNN made {CNN_MOVES_COUNT} moves. Successful were {CNN_SUCCESSFUL_PREDICTION}")
        
        if (CNN_SUCCESSFUL_PREDICTION / CNN_MOVES_COUNT >= CNN_MOVE_PROB):
            CNN_MOVE_PROB = CNN_MOVE_PROB + 0.1
            
        CNN_MOVES_COUNT = 0
        CNN_SUCCESSFUL_PREDICTION = 0


New maximum: 117, after 0 games.
New maximum: 146, after 31 games.
Games: 50, last 500 games avg score: 63.44, avg of successful moves: 9.7, loss 19184.298828125
CNN made 252 moves. Successful were 34
New maximum: 158, after 84 games.
Games: 100, last 500 games avg score: 55.34, avg of successful moves: 8.74, loss 114222.625
CNN made 536 moves. Successful were 44
Games: 150, last 500 games avg score: 61.22, avg of successful moves: 8.78, loss 371735.0625
CNN made 498 moves. Successful were 50
Games: 200, last 500 games avg score: 54.04, avg of successful moves: 8.38, loss 773490.875
CNN made 502 moves. Successful were 48
Games: 250, last 500 games avg score: 61.26, avg of successful moves: 9.06, loss 3736322.25
CNN made 502 moves. Successful were 56
Games: 300, last 500 games avg score: 57.72, avg of successful moves: 8.68, loss 4386056.5
CNN made 527 moves. Successful were 56
New maximum: 178, after 305 games.
Games: 350, last 500 games avg score: 59.74, avg of successful moves: 8.36,

Games: 3100, last 500 games avg score: 55.68, avg of successful moves: 8.82, loss 2286991872.0
CNN made 511 moves. Successful were 63
Games: 3150, last 500 games avg score: 52.3, avg of successful moves: 8.2, loss 5254827520.0
CNN made 522 moves. Successful were 46
Games: 3200, last 500 games avg score: 65.12, avg of successful moves: 9.5, loss 1562847744.0
CNN made 490 moves. Successful were 77
Games: 3250, last 500 games avg score: 62.66, avg of successful moves: 8.64, loss 2204400896.0
CNN made 471 moves. Successful were 47
Games: 3300, last 500 games avg score: 59.52, avg of successful moves: 8.74, loss 2280239360.0
CNN made 514 moves. Successful were 61
Games: 3350, last 500 games avg score: 51.06, avg of successful moves: 8.38, loss 3831833088.0
CNN made 479 moves. Successful were 70
Games: 3400, last 500 games avg score: 59.42, avg of successful moves: 8.86, loss 2990670336.0
CNN made 482 moves. Successful were 68
New maximum: 190, after 3444 games.
Games: 3450, last 500 games a

Games: 6200, last 500 games avg score: 66.78, avg of successful moves: 9.64, loss 1698951168.0
CNN made 503 moves. Successful were 68
Games: 6250, last 500 games avg score: 52.96, avg of successful moves: 8.96, loss 1638192896.0
CNN made 515 moves. Successful were 66
Games: 6300, last 500 games avg score: 65.94, avg of successful moves: 10.04, loss 1265163392.0
CNN made 461 moves. Successful were 74
Games: 6350, last 500 games avg score: 56.3, avg of successful moves: 9.08, loss 1246386432.0
CNN made 499 moves. Successful were 54
Games: 6400, last 500 games avg score: 60.7, avg of successful moves: 9.18, loss 1629839616.0
CNN made 470 moves. Successful were 62
Games: 6450, last 500 games avg score: 61.74, avg of successful moves: 9.12, loss 1228547968.0
CNN made 464 moves. Successful were 68
Games: 6500, last 500 games avg score: 65.48, avg of successful moves: 10.18, loss 407406304.0
CNN made 466 moves. Successful were 73
Games: 6550, last 500 games avg score: 57.0, avg of successful 

Games: 9300, last 500 games avg score: 62.28, avg of successful moves: 10.0, loss 226936144.0
CNN made 490 moves. Successful were 85
Games: 9350, last 500 games avg score: 66.72, avg of successful moves: 9.44, loss 664056448.0
CNN made 502 moves. Successful were 60
Games: 9400, last 500 games avg score: 59.0, avg of successful moves: 9.24, loss 436683712.0
CNN made 456 moves. Successful were 71
Games: 9450, last 500 games avg score: 66.1, avg of successful moves: 9.56, loss 219370368.0
CNN made 483 moves. Successful were 64
Games: 9500, last 500 games avg score: 60.84, avg of successful moves: 9.24, loss 204900160.0
CNN made 489 moves. Successful were 63
Games: 9550, last 500 games avg score: 59.58, avg of successful moves: 8.68, loss 610856960.0
CNN made 499 moves. Successful were 50
Games: 9600, last 500 games avg score: 57.62, avg of successful moves: 8.94, loss 204396688.0
CNN made 461 moves. Successful were 79
Games: 9650, last 500 games avg score: 66.66, avg of successful moves: 

Games: 12400, last 500 games avg score: 57.08, avg of successful moves: 9.3, loss 314763072.0
CNN made 529 moves. Successful were 63
Games: 12450, last 500 games avg score: 54.78, avg of successful moves: 8.48, loss 792332224.0
CNN made 496 moves. Successful were 56
Games: 12500, last 500 games avg score: 64.2, avg of successful moves: 9.48, loss 297743712.0
CNN made 509 moves. Successful were 92
Games: 12550, last 500 games avg score: 54.66, avg of successful moves: 8.76, loss 449759840.0
CNN made 526 moves. Successful were 73
Games: 12600, last 500 games avg score: 57.5, avg of successful moves: 9.26, loss 427969376.0
CNN made 507 moves. Successful were 62
Games: 12650, last 500 games avg score: 64.12, avg of successful moves: 9.42, loss 289542656.0
CNN made 507 moves. Successful were 77
Games: 12700, last 500 games avg score: 57.72, avg of successful moves: 8.44, loss 149844096.0
CNN made 540 moves. Successful were 63
Games: 12750, last 500 games avg score: 54.58, avg of successful 

...

Games: 118700, last 500 games avg score: 41.06, avg of successful moves: 6.92, loss 7451573.0
CNN made 734 moves. Successful were 16
Games: 118750, last 500 games avg score: 47.54, avg of successful moves: 7.24, loss 7517886.5
CNN made 775 moves. Successful were 22
Games: 118800, last 500 games avg score: 44.0, avg of successful moves: 6.92, loss 4116653.0
CNN made 730 moves. Successful were 18
Games: 118850, last 500 games avg score: 52.72, avg of successful moves: 7.9, loss 1947514.75
CNN made 760 moves. Successful were 23
Games: 118900, last 500 games avg score: 42.34, avg of successful moves: 6.86, loss 5253604.0
CNN made 757 moves. Successful were 21
Games: 118950, last 500 games avg score: 54.74, avg of successful moves: 7.98, loss 3578338.75
CNN made 777 moves. Successful were 28
Games: 119000, last 500 games avg score: 45.72, avg of successful moves: 7.14, loss 5861600.5
CNN made 755 moves. Successful were 28
Games: 119050, last 500 games avg score: 43.5, avg of successful move

Games: 121800, last 500 games avg score: 49.44, avg of successful moves: 7.62, loss 1662287.5
CNN made 746 moves. Successful were 27
Games: 121850, last 500 games avg score: 48.44, avg of successful moves: 7.02, loss 2964324.25
CNN made 775 moves. Successful were 20
Games: 121900, last 500 games avg score: 44.14, avg of successful moves: 7.08, loss 5087287.5
CNN made 784 moves. Successful were 30
Games: 121950, last 500 games avg score: 46.0, avg of successful moves: 7.18, loss 3295946.75
CNN made 763 moves. Successful were 22
Games: 122000, last 500 games avg score: 51.5, avg of successful moves: 7.36, loss 3412774.0
CNN made 704 moves. Successful were 18
Games: 122050, last 500 games avg score: 53.08, avg of successful moves: 7.8, loss 1701044.5
CNN made 720 moves. Successful were 29
Games: 122100, last 500 games avg score: 49.74, avg of successful moves: 7.74, loss 5890464.5
CNN made 742 moves. Successful were 25
Games: 122150, last 500 games avg score: 48.44, avg of successful move

Games: 124900, last 500 games avg score: 45.88, avg of successful moves: 7.12, loss 3367471.75
CNN made 756 moves. Successful were 18
Games: 124950, last 500 games avg score: 48.5, avg of successful moves: 7.86, loss 5029027.5
CNN made 771 moves. Successful were 25
Games: 125000, last 500 games avg score: 47.6, avg of successful moves: 7.52, loss 3156116.0
CNN made 758 moves. Successful were 27
Games: 125050, last 500 games avg score: 52.14, avg of successful moves: 7.88, loss 3283852.5
CNN made 753 moves. Successful were 27
Games: 125100, last 500 games avg score: 47.92, avg of successful moves: 7.24, loss 1256969.75
CNN made 747 moves. Successful were 24
Games: 125150, last 500 games avg score: 47.34, avg of successful moves: 7.5, loss 3431591.5
CNN made 763 moves. Successful were 23
Games: 125200, last 500 games avg score: 54.58, avg of successful moves: 7.72, loss 1648712.125
CNN made 747 moves. Successful were 33
Games: 125250, last 500 games avg score: 49.34, avg of successful mo

Games: 128000, last 500 games avg score: 47.18, avg of successful moves: 7.42, loss 6123983.5
CNN made 776 moves. Successful were 19
Games: 128050, last 500 games avg score: 43.56, avg of successful moves: 6.88, loss 2945747.25
CNN made 770 moves. Successful were 24
Games: 128100, last 500 games avg score: 49.46, avg of successful moves: 7.58, loss 1673956.625
CNN made 761 moves. Successful were 21
Games: 128150, last 500 games avg score: 52.96, avg of successful moves: 7.78, loss 4610196.0
CNN made 750 moves. Successful were 28
Games: 128200, last 500 games avg score: 44.68, avg of successful moves: 7.44, loss 2848107.5
CNN made 730 moves. Successful were 28
Games: 128250, last 500 games avg score: 46.06, avg of successful moves: 6.82, loss 7221254.0
CNN made 752 moves. Successful were 19
Games: 128300, last 500 games avg score: 48.44, avg of successful moves: 7.08, loss 6098575.5
CNN made 726 moves. Successful were 13
Games: 128350, last 500 games avg score: 47.66, avg of successful 

CPU times: user 15d 9h 2min 59s, sys: 8h 2min 41s, total: 15d 17h 5min 41s
Wall time: 9d 13h 49min 40s


### Is CNN too small?

In [ ]:
# tensorboard --logdir=./logs --bind_all &

print(MAXIMUM_SCORE)

## Debuggin the Hint

In [207]:
# Total score of one move
reward = 0

# Whether CNN made the move
cnn_made_move_flag = False

# If replay_memory has less than 64 moves, then make a random move
if ((len(replay_memory) < MINIBATCH_SIZE) or (rd.random() > CNN_MOVE_PROB)):
    move = rd.randint(1, ae.ACTIONS_DIMENSION)
else:
    # CNN selects a move
    cnn_made_move_flag = True
    CNN_MOVES_COUNT = CNN_MOVES_COUNT + 1
    scr, move = ae.predict_max_score_3D(field.copy(), Target_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)
    print(f"Predicted move {move} with score of {scr}")

In [208]:
# Make the move
new_field, plate_a, plate_b = ae.make_move_v2_3D(field.copy(), move, ae.MOVES)
field_after_move = new_field.copy()

# Calculate the score and update the field
score, new_field = ae.calculate_score_v2_3D(new_field, plate_a, plate_b)

# If the move is successful, then update the field and check if we have new sets        
successful_move_flag = False

# While we have new sets (thus the score is greater than 0), process them, calculate score and move plates
while (score > 0.):
    if (not successful_move_flag):
        successful_moves = successful_moves + 1
        successful_move_flag = True

    # Add new points to the total score of the move
    reward = reward + score

    # Move plates downward, fill the upper row so, that it doesn't have "easy" sets of three
    # Start from the left lower corner (in order to reuse color_fits())
    new_field = ae.fill_field_3D(new_field)

    # Calculate score and check whether we have new sets
    score, new_field = ae.calculate_score_v2_3D(new_field, (-1, -1), (-1, -1))

print(reward)


0


In [209]:

#
# Train CNN based on the score
#

samples = replay_memory.sample_minibatch(MINIBATCH_SIZE)

# Prepare some things for training
s_before = samples[0]
actions = samples[1]
rewards = samples[2]
s_after = samples[3] 
dones = samples[4]


In [210]:
def loss(model, X, y_true, A):
    prediction = model(X)
    selected_action_values = tf.math.reduce_sum(prediction*A, axis=1)  
    return tf.keras.losses.MSE(y_true, selected_action_values)


def grad(model, inputs, targets, actions):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, actions)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [211]:
rewards_next = np.max(Target_CNN(s_after), axis=1)
actual_values = np.where(dones, rewards, rewards + GAMMA*rewards_next)
selected_actions = tf.one_hot(actions, ae.ACTIONS_DIMENSION)

In [216]:
loss_value, grads = grad(Online_CNN, s_before, actual_values, selected_actions)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))

optimizer.apply_gradients(zip(grads, Online_CNN.trainable_variables))

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss(Online_CNN, s_before, actual_values, selected_actions).numpy()))

Step: 11694, Initial Loss: 20.24236488342285
Step: 11695, Loss: 18.875064849853516


### Now the CNN has been trained.
### Start the long reinforcement-learning cycle

In [ ]:
successful_moves

In [ ]:
new_field = make_move(field, move)
print(new_field)

In [ ]:
temp_field = calculate_score(new_field)

In [ ]:
new_field = np.multiply(new_field, 1.0 - temp_field)

In [ ]:
fill_field(new_field, colors)

In [ ]:
#
# Save model
#
# v1: 20190329, trained on len(replay_memory) = 294912
#aero_cnn.save("Aero_CNN_v1")

## Backup

In [ ]:
#
# Create the moves dictionary
#
moves = {}

for i in range(1, 143):
    old_row, old_column, old_direction = process_move_142(i)
    
    start_row = old_row - 1
    start_col = old_column - 1
    
    if (old_direction == "down"):
        end_row = start_row + 1
        end_col = start_col
    elif (old_direction == "up"):
        end_row = start_row - 1
        end_col = start_col
    elif (old_direction == "right"):
        end_row = start_row
        end_col = start_col + 1
    else:
        end_row = start_row
        end_col = start_col - 1
        
    moves[i] = ((start_row, start_col), (end_row, end_col))
    
print(moves)